# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [245]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [246]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [247]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [248]:
quantities = data.groupby(['CustomerID', 'ProductName'], as_index=False).agg({'Quantity':'sum'})
quantities.head()

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [249]:
quantities_pivot = pd.pivot_table(quantities, index='ProductName', columns='CustomerID', values='Quantity').fillna(0)
quantities_pivot.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [250]:
distances = pd.DataFrame(1/(1 + squareform(pdist(quantities_pivot.T, 'euclidean'))), 
                         index=quantities_pivot.columns, columns=quantities_pivot.columns)

distances.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [251]:
similarities = distances[33].sort_values(ascending=False)[1:]
some_values = []
for k, j in dict(similarities.head()).items():
    some_values.append(k)
some_values

[264, 3535, 3317, 2503, 3305]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [252]:
data5 = quantities.loc[quantities['CustomerID'].isin(some_values)]
data5.head()

,CustomerID,ProductName,Quantity
131,264,Apricots - Halves,1
132,264,Apricots Fresh,1
133,264,Bacardi Breezer - Tropical,1
134,264,Bagel - Plain,1
135,264,Banana - Leaves,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [253]:
data5_new = data5.groupby('ProductName').agg({'Quantity':'sum'}).sort_values(['Quantity'],ascending=False)
data5_new.head()

,Quantity
ProductName,
Butter - Unsalted,3
Wine - Ej Gallo Sierra Valley,3
Towels - Paper / Kraft,3
Soup - Campbells Bean Medley,3
Wine - Blue Nun Qualitatswein,3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [254]:
merged = data5_new.merge(quantities_pivot, on='ProductName')
merged = merged.loc[merged[33]==0].sort_values(['Quantity'],ascending=False)['Quantity'].head()
pd.DataFrame(merged)

,Quantity
ProductName,
Butter - Unsalted,3
Soup - Campbells Bean Medley,3
Wine - Blue Nun Qualitatswein,3
Wine - Ej Gallo Sierra Valley,3
Beans - Wax,2


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [260]:
def recommendations(distance):
    empty = {}
    CustomerIDs = data['CustomerID'].sample(n=6, random_state=1)
                  #[33, 200, 264, 356, 412]
    
    distances = pd.DataFrame(1/(1 + squareform(pdist(quantities_pivot.T, distance))), 
                             index=quantities_pivot.columns, columns=quantities_pivot.columns)
    
    for i in CustomerIDs:
        similarities = distances[i].sort_values(ascending=False)[1:]
        some_values = []
        for k, j in dict(similarities.head()).items():
            some_values.append(k)
        data5 = quantities.loc[quantities['CustomerID'].isin(some_values)]
        data5_new = data5.groupby('ProductName').agg({'Quantity':'sum'}).sort_values(['Quantity'],ascending=False)
        merged = data5_new.merge(quantities_pivot, on='ProductName')
        empty[i] = merged.loc[merged[33]==0].sort_values(['Quantity'],ascending=False)[[i,'Quantity']].head().index

    return empty

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [261]:
pd.DataFrame.from_dict(recommendations('euclidean'))

,94910,639,51834,69931,53214,81981
0,Halibut - Steaks,Cod - Black Whole Fillet,Table Cloth 81x81 White,Pernod,"Fish - Scallops, Cold Smoked",Cheese - Victor Et Berthold
1,Scampi Tail,Wine - Ej Gallo Sierra Valley,Cocoa Butter,Milk - 1%,Juice - Orange,Brandy - Bar
2,Dc Hikiage Hira Huba,Yoghurt Tubes,Muffin - Zero Transfat,Carbonated Water - Blackcherry,Ocean Spray - Ruby Red,Soupcontfoam16oz 116con
3,Onions - Cippolini,Halibut - Steaks,Beets - Mini Golden,Bacardi Breezer - Tropical,Carbonated Water - Cherry,Dc Hikiage Hira Huba
4,"Cheese - Boursin, Garlic / Herbs","Wine - Magnotta, Merlot Sr Vqa",Olives - Stuffed,Squid U5 - Thailand,Sauerkraut,"Lentils - Red, Dry"


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [262]:
distances = ['correlation','cityblock','cosine','jaccard', 'chebyshev', 'rogerstanimoto', 'sokalmichener']
for i in distances:
    display(pd.DataFrame.from_dict(recommendations(i)))

,94910,639,51834,69931,53214,81981
0,Halibut - Steaks,"Seedlings - Mix, Organic",Wiberg Super Cure,Wine - Ruffino Chianti,Cake - Mini Cheesecake,Cheese - Victor Et Berthold
1,Longos - Chicken Wings,Wine - Toasted Head,Wine - Redchard Merritt,Sardines,Coffee - Dark Roast,Wanton Wrap
2,Cod - Black Whole Fillet,Juice - Apple Cider,Bread - Raisin Walnut Oval,Squid U5 - Thailand,Fuji Apples,Whmis - Spray Bottle Trigger
3,Smirnoff Green Apple Twist,Garlic,"Mushrooms - Black, Dried",Beans - Kidney White,"Fish - Scallops, Cold Smoked",Puree - Passion Fruit
4,Snapple - Iced Tea Peach,Dried Figs,Lettuce - California Mix,Cookies Cereal Nut,Mayonnaise - Individual Pkg,Beef - Texas Style Burger


,94910,639,51834,69931,53214,81981
0,Skirt - 29 Foot,Yoghurt Tubes,Knife Plastic - White,Ice Cream Bar - Hageen Daz To,Bar - Granola Trail Mix Fruit Nut,Pork - Kidney
1,Bar - Granola Trail Mix Fruit Nut,Halibut - Steaks,Sauce - Hollandaise,"Wine - White, Colubia Cresh",Pears - Bosc,Olives - Stuffed
2,Beef Ground Medium,Ice Cream Bar - Hageen Daz To,Ice Cream Bar - Hageen Daz To,Pork - Kidney,Sausage - Breakfast,Cod - Black Whole Fillet
3,"Tart Shells - Sweet, 4",Bacardi Breezer - Tropical,Bar - Granola Trail Mix Fruit Nut,Wine - Chardonnay South,Flour - Pastry,Berry Brulee
4,Rice - Jasmine Sented,Butter - Unsalted,"Sole - Dover, Whole, Fresh",Muffin - Carrot Individual Wrap,Scallop - St. Jaques,Beef - Texas Style Burger


,94910,639,51834,69931,53214,81981
0,Bread Crumbs - Japanese Style,Juice - Apple Cider,Wiberg Super Cure,Wine - Ruffino Chianti,"Fish - Scallops, Cold Smoked",Wanton Wrap
1,Longos - Grilled Salmon With Bbq,"Seedlings - Mix, Organic",Wine - Redchard Merritt,Sardines,Coffee - Dark Roast,Puree - Passion Fruit
2,Appetizer - Mushroom Tart,Salmon - Sockeye Raw,Bread - Raisin Walnut Oval,Squid U5 - Thailand,Cake - Mini Cheesecake,Beef - Texas Style Burger
3,Smirnoff Green Apple Twist,Wine - Pinot Noir Latour,"Mushrooms - Black, Dried",Beans - Kidney White,Mayonnaise - Individual Pkg,Longos - Chicken Wings
4,Peas - Frozen,Cheese - Parmesan Cubes,Lettuce - California Mix,Cookies Cereal Nut,Fuji Apples,Cheese - Victor Et Berthold


,94910,639,51834,69931,53214,81981
0,"Garlic - Primerba, Paste","Mushrooms - Black, Dried",Turkey - Oven Roast Breast,"Water - Mineral, Natural","Sole - Dover, Whole, Fresh","Water, Tap"
1,Berry Brulee,Fenngreek Seed,Ecolab - Mikroklene 4/4 L,"Pepsi - Diet, 355 Ml",Truffle Cups - Brown,"Oil - Shortening,liqud, Fry"
2,Sage - Ground,Soup Knorr Chili With Beans,"Cheese - Boursin, Garlic / Herbs",Lettuce - Frisee,Appetizer - Mushroom Tart,"Mushroom - Porcini, Dry"
3,"Thyme - Lemon, Fresh",Bacardi Breezer - Tropical,Cake - Mini Cheesecake,Coffee - Irish Cream,Lime Cordial - Roses,"Chocolate - Semi Sweet, Calets"
4,"Placemat - Scallop, White",Sardines,"Appetizer - Mini Egg Roll, Shrimp","Coconut - Shredded, Sweet",Cheese Cloth No 100,Beans - Kidney White


,94910,639,51834,69931,53214,81981
0,Dried Figs,"Wine - White, Colubia Cresh",Browning Caramel Glace,Juice - Apple Cider,Eggplant - Asian,"Cheese - Brie, Triple Creme"
1,"Rosemary - Primerba, Paste",Beef - Top Sirloin,Wine - Fume Blanc Fetzer,Hinge W Undercut,Juice - V8 Splash,Chocolate - Feathers
2,Ocean Spray - Kiwi Strawberry,Mussels - Cultivated,Ecolab - Mikroklene 4/4 L,Crush - Cream Soda,"Wine - Red, Harrow Estates, Cab",Langers - Ruby Red Grapfruit
3,Sea Bass - Whole,Ezy Change Mophandle,Eggplant - Asian,Ice Cream Bar - Hageen Daz To,Beans - Kidney White,Jagermeister
4,"Soup - Campbells, Lentil",Scampi Tail,Flavouring - Orange,"Salmon - Atlantic, Skin On","Potatoes - Instant, Mashed",Wiberg Super Cure


,94910,639,51834,69931,53214,81981
0,Vinegar - Sherry,Wine - Toasted Head,Bandage - Fexible 1x3,Cod - Black Whole Fillet,"Fish - Scallops, Cold Smoked",Beef Wellington
1,Longos - Grilled Salmon With Bbq,Tia Maria,Pastry - Choclate Baked,Soupcontfoam16oz 116con,Cheese - Parmesan Cubes,Soup Knorr Chili With Beans
2,Skirt - 29 Foot,Pastry - Butterscotch Baked,Veal - Inside,Banana - Leaves,Duck - Breast,Rice - Long Grain
3,Juice - Orange,"Lemonade - Natural, 591 Ml",Hot Chocolate - Individual,Puree - Mocha,Mayonnaise - Individual Pkg,Rambutan
4,Cheese - Parmesan Cubes,Bandage - Fexible 1x3,"Wine - Red, Colio Cabernet",Garbag Bags - Black,Fuji Apples,Sunflower Seed Raw


,94910,639,51834,69931,53214,81981
0,Vinegar - Sherry,Wine - Toasted Head,Bandage - Fexible 1x3,Cod - Black Whole Fillet,"Fish - Scallops, Cold Smoked",Beef Wellington
1,Longos - Grilled Salmon With Bbq,Tia Maria,Pastry - Choclate Baked,Soupcontfoam16oz 116con,Cheese - Parmesan Cubes,Soup Knorr Chili With Beans
2,Skirt - 29 Foot,Pastry - Butterscotch Baked,Veal - Inside,Banana - Leaves,Duck - Breast,Rice - Long Grain
3,Juice - Orange,"Lemonade - Natural, 591 Ml",Hot Chocolate - Individual,Puree - Mocha,Mayonnaise - Individual Pkg,Rambutan
4,Cheese - Parmesan Cubes,Bandage - Fexible 1x3,"Wine - Red, Colio Cabernet",Garbag Bags - Black,Fuji Apples,Sunflower Seed Raw
